In [2]:
# importing packages

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [3]:
# instantiating model

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

D:\anaconda3\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nisha\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
# encoding and calculating sentiment

tokens = tokenizer.encode('this is so good but its disgusting', return_tensors='pt')

result = model(tokens)
int(torch.argmax(result.logits))+1

3

In [32]:
# collect reviews

r = requests.get("https://www.yelp.com/biz/masuya-japanese-seafood-restaurant-sydney?osq=Japanese")
soup = BeautifulSoup(r.text, 'html.parser')
regex =  re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

reviews[0]

"We visited this restaurant for dinner during the Vivid Sydney week. Some other Japanese restaurants charged minimum $45 per person during this time, but luckily we found this restaurant which satisfied our cravings for sushi and sashimi.Food: We ordered the blue fin tuna sushi/sashimi platter for two and a drink. The fish was fresh and the sushi taste great. The drink was similar to Starbucks green tea frappucino. But there are a wide variety of sake options and non-alcoholic drinks. Service: We just walked in without knowing that we need to make a reservation during the Vivid Sydney week. The staff was nice and seated us at the bar. The service was quick and friendly.Ambiance: The restaurant is classy and casual. Reservation is recommended especially during weekends and special events. Price: The price is mid-range, not as expensive as other Japanese restaurants. There are wide variety of sushi and sashimi platters, so you can choose the ones that fit your budget.Conclusion: It's an 

In [33]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.array(reviews), columns=['review'])
df['review'].iloc[0]

"We visited this restaurant for dinner during the Vivid Sydney week. Some other Japanese restaurants charged minimum $45 per person during this time, but luckily we found this restaurant which satisfied our cravings for sushi and sashimi.Food: We ordered the blue fin tuna sushi/sashimi platter for two and a drink. The fish was fresh and the sushi taste great. The drink was similar to Starbucks green tea frappucino. But there are a wide variety of sake options and non-alcoholic drinks. Service: We just walked in without knowing that we need to make a reservation during the Vivid Sydney week. The staff was nice and seated us at the bar. The service was quick and friendly.Ambiance: The restaurant is classy and casual. Reservation is recommended especially during weekends and special events. Price: The price is mid-range, not as expensive as other Japanese restaurants. There are wide variety of sushi and sashimi platters, so you can choose the ones that fit your budget.Conclusion: It's an 

In [34]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [35]:
sentiment_score(df['review'].iloc[0])

4

In [36]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x))
df

,review,sentiment
0,We visited this restaurant for dinner during t...,4
1,Wonderfully fresh Sashimi and the tempura soft...,5
2,This has to be one of the top japanese restaur...,4
3,Overall I'd give it a 4.5 but rounding up. We ...,4
4,"Welcome to the new Japanese McDonald.Noisy, br...",4
5,Nice restaurant with genuine japanese cuisine....,4
6,GREAT great Japanese in Sydney. I came here af...,5
7,Went there late. Very nice friendly stuffs. Fo...,5
8,Had a lovely meal here. Service was efficient...,5
9,Not exactly sure how long this place has been ...,4
